<a href="https://colab.research.google.com/github/Amisha1019/Gen-AI-Customer-Service-Bot-Internship-Task/blob/main/Task_3_of_Gen_Ai_chatbot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's generate some code to address the notebook rendering error on GitHub. This code will read your `.ipynb` file, check for the `state` structure, and either add the missing `state` key or remove the `widgets` entry if it's malformed, based on the error message you received.

In [42]:
import json
import os

notebook_path = 'YOUR_NOTEBOOK_NAME.ipynb' # Replace with the actual path to your notebook file

if not os.path.exists(notebook_path):
    print(f"Error: Notebook file not found at {notebook_path}")
else:
    try:
        with open(notebook_path, 'r', encoding='utf-8') as f:
            notebook_content = json.load(f)

        # Check and fix the state structure
        if 'metadata' in notebook_content and 'widgets' in notebook_content['metadata']:
            widgets_metadata = notebook_content['metadata']['widgets']
            if isinstance(widgets_metadata, dict):
                if 'state' not in widgets_metadata:
                    print("Adding missing 'state' key to state")
                    widgets_metadata['state'] = {}
                # You might want to add more robust checks here if other parts of the widgets structure are often broken
            else:
                print("state is not a dictionary, removing it to fix potential rendering issues.")
                del notebook_content['metadata']['widgets']
        else:
             print("state not found, no changes needed for this specific error.")


        # Save the modified notebook
        output_notebook_path = 'fixed_' + notebook_path
        with open(output_notebook_path, 'w', encoding='utf-8') as f:
            json.dump(notebook_content, f, indent=1)

        print(f"Notebook processing complete. Modified notebook saved as {output_notebook_path}")
        print("Please upload this fixed notebook to your GitHub profile.")

    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {notebook_path}. Make sure it's a valid .ipynb file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Error: Notebook file not found at YOUR_NOTEBOOK_NAME.ipynb


In [27]:
!pip install pandas numpy sentence-transformers faiss-cpu streamlit

In [28]:
!pip install spacy python -m spacy download en_core_web_sm


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -m


In [29]:
import pandas as pd
import numpy as np
import os
import glob
import faiss
import pickle
import xml.etree.ElementTree as ET
from sentence_transformers import SentenceTransformer

In [30]:
DATA_DIR = '/0000001_1.xml'
INDEX_PATH = 'medquad_faiss.index'
DB_PATH = 'medquad_db.pkl'
EMBEDDING_MODEL = 'all-MiniLM-L6-v2'

In [31]:
def parse_medquad_xml(file_path):
    """
    Parses a single MedQuAD XML file to extract Question-Answer pairs.
    """
    qa_list = []
    try:
        tree = ET.parse('/0000001_1.xml')
        root = tree.getroot()

        for qapair in root.findall('/0000001_1.xml'):
            question_element = qapair.find('Question')
            answer_element = qapair.find('Answer')

            if question_element is not None and answer_element is not None:
                question = question_element.text.strip()
                answer = answer_element.text.strip()


                focus_element = root.find('Focus')
                focus = focus_element.text.strip() if focus_element is not None else "N/A"

                qa_list.append({
                    'Question': question,
                    'Answer': answer,
                    'Focus': focus
                })
    except Exception as e:
        print(f"Error parsing {'/0000001_1.xml'}: {e}")

    return qa_list

In [32]:
def parse_medquad_xml(file_path):
    """
    Parses a single MedQuAD XML file to extract Question-Answer pairs.
    """
    qa_list = []
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()


        for qapair in root.findall('.//QAPair'):

            question_element = qapair.find('.//Question')
            answer_element = qapair.find('.//Answer')

            if question_element is not None and answer_element is not None:
                question = question_element.text.strip()
                answer = answer_element.text.strip()


                focus_element = qapair.find('.//Focus')
                focus = focus_element.text.strip() if focus_element is not None else "N/A"

                qa_list.append({
                    'Question': question,
                    'Answer': answer,
                    'Focus': focus
                })
    except Exception as e:
        print(f"Error parsing {file_path}: {e}")

    return qa_list

def create_knowledge_base():
    """
    Loads data, creates embeddings, and builds the FAISS index.
    """
    print("Starting Knowledge Base Creation...")


    xml_files = glob.glob(os.path.join(DATA_DIR))
    if not xml_files:
        print(f"ERROR: No XML files found in {DATA_DIR}. Please check the path.")
        return

    all_qa_data = []
    for xml_file in xml_files:
        all_qa_data.extend(parse_medquad_xml(xml_file))

    qa_df = pd.DataFrame(all_qa_data)


    qa_df = qa_df.dropna(subset=['Question', 'Answer'])
    qa_df = qa_df.drop_duplicates(subset=['Answer']).reset_index(drop=True)

    print(f"Total cleaned Q&A pairs loaded: {len(qa_df):,}")
    if qa_df.empty:
        print("No data found after cleaning. Exiting.")
        return

    print(f"Loading Sentence Transformer model: {EMBEDDING_MODEL}...")
    model = SentenceTransformer(EMBEDDING_MODEL)


    questions = qa_df['Question'].tolist()
    print("Generating embeddings for questions...")
    embeddings = model.encode(questions, show_progress_bar=True, convert_to_numpy=True)


    print("Building FAISS index...")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    print(f"FAISS index contains {index.ntotal} vectors.")


    print(f"Saving FAISS index to {INDEX_PATH}...")
    faiss.write_index(index, INDEX_PATH)

    print(f"Saving knowledge base (DataFrame) to {DB_PATH}...")
    with open(DB_PATH, 'wb') as f:
        pickle.dump(qa_df, f)

    print("Knowledge Base Creation Complete.")

create_knowledge_base()

Starting Knowledge Base Creation...
Total cleaned Q&A pairs loaded: 7
Loading Sentence Transformer model: all-MiniLM-L6-v2...
Generating embeddings for questions...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Building FAISS index...
FAISS index contains 7 vectors.
Saving FAISS index to medquad_faiss.index...
Saving knowledge base (DataFrame) to medquad_db.pkl...
Knowledge Base Creation Complete.


In [33]:
def create_knowledge_base():
    """
    Loads data, creates embeddings, and builds the FAISS index.
    """
    print("Starting Knowledge Base Creation...")


    xml_files = glob.glob(os.path.join(DATA_DIR, '/0000001_1.xml'))
    if not xml_files:
        print(f"ERROR: No XML files found in {DATA_DIR}. Please check the path.")
        return

    all_qa_data = []
    for xml_file in xml_files:
        all_qa_data.extend(parse_medquad_xml('/0000001_1.xml'))

    qa_df = pd.DataFrame(all_qa_data)


    qa_df = qa_df.dropna(subset=['Question', 'Answer'])
    qa_df = qa_df.drop_duplicates(subset=['Answer']).reset_index(drop=True)

    print(f"Total cleaned Q&A pairs loaded: {len(qa_df):,}")
    if qa_df.empty:
        return

    print(f"Loading Sentence Transformer model: {EMBEDDING_MODEL}...")
    model = SentenceTransformer(EMBEDDING_MODEL)


    questions = qa_df['Question'].tolist()
    print("Generating embeddings for questions...")
    embeddings = model.encode(questions, show_progress_bar=True, convert_to_numpy=True)


    print("Building FAISS index...")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    print(f"FAISS index contains {index.ntotal} vectors.")


    print(f"Saving FAISS index to {INDEX_PATH}...")
    faiss.write_index(index, INDEX_PATH)

    print(f"Saving knowledge base (DataFrame) to {DB_PATH}...")
    with open(DB_PATH, 'wb') as f:
        pickle.dump(qa_df, f)

    print("Knowledge Base Creation Complete.")

create_knowledge_base()

Starting Knowledge Base Creation...
Total cleaned Q&A pairs loaded: 7
Loading Sentence Transformer model: all-MiniLM-L6-v2...
Generating embeddings for questions...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Building FAISS index...
FAISS index contains 7 vectors.
Saving FAISS index to medquad_faiss.index...
Saving knowledge base (DataFrame) to medquad_db.pkl...
Knowledge Base Creation Complete.


Now that the knowledge base is created, we can implement a function to query it. This function will take a user query, embed it using the same model, and then use the FAISS index to find the most similar questions in the knowledge base. Finally, it will retrieve and display the corresponding answers.

In [34]:
def load_knowledge_base():
    """
    Loads the FAISS index and the knowledge base (DataFrame) from disk.
    """
    print("Loading knowledge base and index...")
    index = faiss.read_index(INDEX_PATH)
    with open(DB_PATH, 'rb') as f:
        qa_df = pickle.load(f)
    print("Knowledge base and index loaded.")
    return index, qa_df

def query_knowledge_base(query, index, qa_df, model, k=5):
    """
    Queries the knowledge base with a given query.

    Args:
        query (str): The user's query.
        index (faiss.Index): The FAISS index.
        qa_df (pd.DataFrame): The DataFrame containing the Q&A pairs.
        model (SentenceTransformer): The sentence embedding model.
        k (int): The number of nearest neighbors to retrieve.

    Returns:
        list: A list of dictionaries containing the top k results (Question, Answer, Score).
    """

    query_embedding = model.encode([query], convert_to_numpy=True)


    distances, indices = index.search(query_embedding, k)


    results = []
    for i in range(k):
        qa_index = indices[0][i]
        distance = distances[0][i]
        result = {
            'Question': qa_df.loc[qa_index, 'Question'],
            'Answer': qa_df.loc[qa_index, 'Answer'],
            'Focus': qa_df.loc[qa_index, 'Focus'],
            'Score': 1 - (distance / index.reconstruct(0).dot(index.reconstruct(0))) # Cosine similarity from L2 distance
        }
        results.append(result)

    return results


try:
    faiss_index, knowledge_base_df = load_knowledge_base()
    embedding_model = SentenceTransformer(EMBEDDING_MODEL)
except FileNotFoundError:
    print("Knowledge base files not found. Please run the knowledge base creation step first.")
    faiss_index = None
    knowledge_base_df = None
    embedding_model = None


if faiss_index is not None and knowledge_base_df is not None and embedding_model is not None:
    user_query = "What are the symptoms of diabetes?"
    print(f"\nSearching for: '{user_query}'")
    search_results = query_knowledge_base(user_query, faiss_index, knowledge_base_df, embedding_model)

    print("\nSearch Results:")
    for i, result in enumerate(search_results):
        print(f"--- Result {i+1} ---")
        print(f"Question: {result['Question']}")
        print(f"Answer: {result['Answer']}")
        print(f"Focus: {result['Focus']}")
        print(f"Score: {result['Score']:.4f}")

Loading knowledge base and index...
Knowledge base and index loaded.

Searching for: 'What are the symptoms of diabetes?'

Search Results:
--- Result 1 ---
Question: What are the symptoms of Adult Acute Lymphoblastic Leukemia ?
Answer: Signs and symptoms of adult ALL include fever, feeling tired, and easy bruising or bleeding. The early signs and symptoms of ALL may be like the flu or other common diseases. Check with your doctor if you have any of the following:         - Weakness or feeling tired.    -  Fever or night sweats.    - Easy bruising or bleeding.    -  Petechiae (flat, pinpoint spots under the skin, caused by bleeding).    - Shortness of breath.    - Weight loss or loss of appetite.    - Pain in the bones or stomach.    - Pain or feeling of fullness below the ribs.    - Painless lumps in the neck, underarm, stomach, or groin.    - Having many infections.        These and other signs and symptoms may be caused by adult acute lymphoblastic leukemia or by other conditions.
Fo

In [35]:
import streamlit as st
import faiss
import pickle
import numpy as np
import spacy
from sentence_transformers import SentenceTransformer
import os

In [36]:
INDEX_PATH = 'medquad_faiss.index'
DB_PATH = 'medquad_db.pkl'

@st.cache_resource
def load_rag_components():
    """Load the Sentence Transformer model, FAISS index, and Q&A database."""
    try:

        with open('model_name.txt', 'r') as f:
            model_name = f.read().strip()


        model = SentenceTransformer(model_name)
        index = faiss.read_index(INDEX_PATH)
        qa_df = pd.read_pickle(DB_PATH)

        print("RAG components loaded successfully.")
        return model, index, qa_df
    except Exception as e:
        st.error(f"Error loading RAG components. Did you run knowledge_base.py first? Error: {e}")
        return None, None, None



In [37]:
@st.cache_resource
def load_ner_model():
    """Load the SpaCy model for Named Entity Recognition."""
    try:

        nlp = spacy.load("en_core_web_sm")
        print("SpaCy NER model loaded.")
        return nlp
    except Exception as e:
        st.warning(f"Could not load SpaCy model. NER will be disabled. Error: {e}")
        return None

In [38]:
def semantic_search(query, model, index, qa_df, k=3):
    """Encodes the query and searches the FAISS index for the top k matches."""

    query_embedding = model.encode([query])

    faiss.normalize_L2(query_embedding)
    query_embedding = query_embedding.astype('float32')


    distances, indices = index.search(query_embedding, k)


    results = []
    for i in range(k):
        idx = indices[0][i]
        score = distances[0][i]

        if idx >= 0:
            results.append({
                'score': score,
                'Question': qa_df.iloc[idx]['Question'],
                'Answer': qa_df.iloc[idx]['Answer'],
                'Focus': qa_df.iloc[idx]['Focus']
            })
    return results

In [39]:
def perform_ner(query, nlp):
    """Identifies medical entities (simulated) in the user query."""
    if nlp is None:
        return []

    doc = nlp(query)
    entities = []


    for ent in doc.ents:

        if ent.label_ in ['PERSON', 'ORG', 'NORP', 'PRODUCT', 'EVENT']:
            entities.append((ent.text, ent.label_))


    medical_keywords = {
        'symptoms': ['pain', 'fever', 'tired', 'cough', 'bleeding'],
        'diseases': ['cancer', 'leukemia', 'glaucoma', 'myeloid', 'lymphoblastic'],
        'treatments': ['chemotherapy', 'radiation', 'transplant', 'surgery', 'tyrosine kinase']
    }

    query_lower = query.lower()
    for entity_type, keywords in medical_keywords.items():
        for keyword in keywords:
            if keyword in query_lower and (keyword, entity_type.upper()) not in entities:
                 entities.append((keyword.capitalize(), entity_type.upper()))

    return entities

In [40]:
st.set_page_config(page_title="MedQuAD Medical Chatbot 🩺", layout="wide")
st.title("MedQuAD Medical Chatbot 🩺")
st.markdown("---")


model, index, qa_df = load_rag_components()
nlp = load_ner_model()

if model and index and not qa_df.empty:
    if "messages" not in st.session_state:
        st.session_state.messages = [
            {"role": "assistant", "content": "Hello! I am a medical question-answering bot powered by the MedQuAD dataset. Ask me about diseases, symptoms, or treatments."}
        ]


    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

2025-10-30 08:20:15.715 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:15.730 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:15.733 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:15.735 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:15.736 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:15.740 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:15.742 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:15.745 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

SpaCy NER model loaded.


In [41]:
if prompt := st.chat_input("Ask a medical question..."):

        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        with st.chat_message("assistant"):
            with st.spinner("Searching and generating response..."):


                search_results = semantic_search(prompt, model, index, qa_df, k=3)
                best_match = search_results[0]


                entities = perform_ner(prompt, nlp)



                # A. NER Display
                entity_report = ""
                if entities:
                    entity_report += "**Identified Medical Entities:** "
                    entity_report += ", ".join([f"**{text}** (*{label}*)" for text, label in entities])

                # B. Retrieval Summary & Answer
                if best_match['score'] > 0.5:

                    answer_content = f"### Answer (Source: {best_match['Focus']})\n{best_match['Answer']}\n\n"
                    answer_content += f"*Retrieval Confidence Score: {best_match['score']:.4f} (from matching question: {best_match['Question'][:80]}...)*"
                else:
                    answer_content = "I could not find a highly relevant answer in my knowledge base. Please try rephrasing your question."


                st.markdown(entity_report)
                st.markdown("---")
                st.markdown(answer_content)


                full_assistant_response = f"{entity_report}\n\n---\n\n{answer_content}"
                st.session_state.messages.append({"role": "assistant", "content": full_assistant_response})

else:
    st.warning("Please run `python knowledge_base.py` first to generate the necessary index and database files.")

2025-10-30 08:20:17.884 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:17.889 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:17.893 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:17.895 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:17.898 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:17.899 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:17.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:20:17.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar